In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
# Load the dataset
df = pd.read_csv('malicious_phish.csv')

In [3]:
# Categorize the classes
df['type'] = df['type'].replace({'benign': 0, 'defacement': 1, 'phishing': 2, 'malware': 3})


In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['url'], df['type'], test_size=0.2, random_state=0)

In [5]:
# Encode the labels
onehot_encoder = OneHotEncoder(sparse=False)
y_train = onehot_encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test = onehot_encoder.transform(y_test.values.reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
# Tokenize the URLs
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [7]:
# Pad the sequences
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=200, padding='post', truncating='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=200, padding='post', truncating='post')

In [8]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=200),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           14883424  
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 64)                409664    
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 15295300 (58.35 MB)
Trainable params: 15295300 (58.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [10]:
# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5
16280/16280 [==============================] - 107s 6ms/step - loss: 0.1799 - accuracy: 0.9373 - val_loss: 0.1406 - val_accuracy: 0.9511
Epoch 2/5
16280/16280 [==============================] - 88s 5ms/step - loss: 0.1506 - accuracy: 0.9506 - val_loss: 0.1483 - val_accuracy: 0.9509
Epoch 3/5
16280/16280 [==============================] - 87s 5ms/step - loss: 0.1510 - accuracy: 0.9518 - val_loss: 0.1617 - val_accuracy: 0.9491
Epoch 4/5
16280/16280 [==============================] - 85s 5ms/step - loss: 0.1524 - accuracy: 0.9527 - val_loss: 0.1597 - val_accuracy: 0.9524
Epoch 5/5
16280/16280 [==============================] - 88s 5ms/step - loss: 0.1483 - accuracy: 0.9540 - val_loss: 0.1468 - val_accuracy: 0.9533


In [11]:
# Evaluate the model
model.evaluate(X_test, y_test)

4070/4070 [==============================] - 10s 2ms/step - loss: 0.1468 - accuracy: 0.9533


[0.1467837244272232, 0.9533396363258362]